In [1]:
import numpy as np
import pickle
import cv2, os
from glob import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [3]:
def get_image_size():
	img = cv2.imread('gestures/1/100.jpg', 0)
	return img.shape

def get_num_of_classes():
	return len(glob('gestures/*'))

image_x, image_y = get_image_size()

In [4]:
print(get_image_size())
print(get_num_of_classes())

(50, 50)
7


In [5]:
def cnn_model():
    num_of_classes = get_num_of_classes()
    model = keras.Sequential()
    model.add(layers.Conv2D(16,(2,2), input_shape=(image_x,image_y,1), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(layers.Conv2D(32,(3,3), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(3,3),strides=(3,3),padding='same'))
    model.add(layers.Conv2D(64,(5,5), activation="relu"))
    model.add(layers.MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(layers.Flatten())
    model.add(layers.Dense(128,activation="relu"))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(num_of_classes,activation="softmax"))
    model.summary()
    model.compile(
        optimizer = keras.optimizers.SGD(learning_rate=0.01),
        loss = keras.losses.CategoricalCrossentropy(),
        metrics=['accuracy']
    )
    filepath="cnn_model_keras2.h5"
    checkpoint1 = tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor='val_accuracy',
        verbose=1, save_best_only=True, mode='max'
    )
    callbacks_list = [checkpoint1]
    tf.keras.utils.plot_model(model=model, to_file="model.png", show_shapes=True)
    return model,callbacks_list  
cnn_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 49, 49, 16)        80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 25, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 23, 23, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 64)          51264     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 1, 1, 64)         0

(<keras.engine.sequential.Sequential at 0x20fd492ac70>,
 [<keras.callbacks.ModelCheckpoint at 0x20fd4948b20>])

In [6]:
with open("train_images.pkl", "rb") as f:
    train_images = np.array(pickle.load(f))
with open("train_labels.pkl", "rb") as f:
    train_labels = np.array(pickle.load(f), dtype=np.int32)

with open("val_images.pkl", "rb") as f:
    val_images = np.array(pickle.load(f))
with open("val_labels.pkl", "rb") as f:
    val_labels = np.array(pickle.load(f), dtype=np.int32)
print(val_labels.shape)
print(train_labels.shape)
print(val_images.shape)
print(train_images.shape)
print("change")
train_images = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
val_images = np.reshape(val_images, (val_images.shape[0], image_x, image_y, 1))
train_labels = tf.keras.utils.to_categorical(train_labels)
val_labels = tf.keras.utils.to_categorical(val_labels)
    
print(val_labels.shape)
print(train_labels.shape)
print(val_images.shape)
print(train_images.shape)


(1400,)
(14000,)
(1400, 50, 50)
(14000, 50, 50)
change
(1400, 7)
(14000, 7)
(1400, 50, 50, 1)
(14000, 50, 50, 1)


In [7]:
model, callbacks_list = cnn_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 49, 49, 16)        80        
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 25, 25, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 23, 23, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 64)          51264     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 1, 1, 64)        

In [8]:
model.fit(train_images, train_labels, 
    validation_data=(val_images, val_labels), 
    epochs=15, batch_size=500, callbacks=callbacks_list)
scores = model.evaluate(val_images, val_labels, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))
    

tf.keras.backend.clear_session();

Epoch 1/15
28/28 [==============================] - ETA: 0s - loss: 10.3242 - accuracy: 0.2066
Epoch 1: val_accuracy improved from -inf to 0.38929, saving model to cnn_model_keras2.h5
28/28 [==============================] - 29s 1s/step - loss: 10.3242 - accuracy: 0.2066 - val_loss: 1.6578 - val_accuracy: 0.3893
Epoch 2/15
28/28 [==============================] - ETA: 0s - loss: 1.0733 - accuracy: 0.6374
Epoch 2: val_accuracy improved from 0.38929 to 0.96000, saving model to cnn_model_keras2.h5
28/28 [==============================] - 30s 1s/step - loss: 1.0733 - accuracy: 0.6374 - val_loss: 0.2575 - val_accuracy: 0.9600
Epoch 3/15
28/28 [==============================] - ETA: 0s - loss: 0.1600 - accuracy: 0.9654
Epoch 3: val_accuracy improved from 0.96000 to 0.99714, saving model to cnn_model_keras2.h5
28/28 [==============================] - 32s 1s/step - loss: 0.1600 - accuracy: 0.9654 - val_loss: 0.0249 - val_accuracy: 0.9971
Epoch 4/15
28/28 [==============================] - ETA: